# Babyloania Transaction Data 

In [20]:
import os
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns


%matplotlib inline

In [72]:
raw_2017 = pd.read_excel("../data/pesanan_dibuat_1-Jan-2017_sampai_31-Dec-2017.xlsx", parse_dates=['Tanggal Pesanan','Mulai Sewa','Selesai Sewa'])
# converters={['Tanggal_Pesanan','Mulai_Sewa','Selesai_Sewa']: pd.to_datetime}
data_2017 = raw_2017.drop(columns=['Nomor', 'Nama Pelanggan', 'Alamat Pelanggan (ulangi)','Telepon', 'Alamat Kirim', 'Email', 'Lunas',
       'Total Biaya', 'Jumlah Dibayar', 'Catatan','Varian',
       'Warna', 'Status'])
data_2017 = data_2017.rename(lambda x: x.replace(' ', '_'), axis=1) #Removing spaces in column names
data_2017 = data_2017.rename(columns={'Nomor(ulang)': 'Nomor',
                                      'Nama_Pelanggan_(ulangi)': 'Nama_Pelanggan',
                                      'Pemilik_Unit,_Skema_Bagi_Hasil': 'Pemilik_Unit', 
                                      'Unnamed:_23': 'Skema_Bagi_Hasil'})



In [61]:
raw_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7642 entries, 0 to 7641
Data columns (total 24 columns):
Nomor                             3237 non-null object
Nama Pelanggan                    3237 non-null object
Telepon                           3237 non-null object
Alamat Kirim                      3237 non-null object
Email                             3232 non-null object
Lunas                             3237 non-null object
Total Biaya                       3237 non-null float64
Jumlah Dibayar                    3237 non-null float64
Catatan                           983 non-null object
Tanggal Pesanan                   7630 non-null datetime64[ns]
Nomor(ulang)                      7630 non-null object
Komponen Biaya                    7630 non-null object
Harga                             7630 non-null float64
Kode Unit                         4063 non-null object
Varian                            280 non-null object
Warna                             2338 non-null object
Mula

In [74]:
raw_2018 = pd.read_excel("../data/pesanan_dibuat_1-Jan-2018_sampai_31-Jan-2020.xlsx", parse_dates=['Tanggal Pesanan','Mulai Sewa','Selesai Sewa'])
data_2018 = raw_2018.drop(columns=['Nomor', 'Nama Pelanggan', 'Alamat Pelanggan (ulangi)','Telepon', 'Alamat Kirim', 'Email', 'Lunas',
       'Total Biaya', 'Jumlah Dibayar', 'Catatan','Varian',
       'Warna', 'Status'])
data_2018 = data_2018.rename(lambda x: x.replace(' ', '_'), axis=1) #Removing spaces in column names
data_2018 = data_2018.rename(columns={'Nomor(ulang)': 'Nomor',
                                      'Nama_Pelanggan_(ulangi)': 'Nama_Pelanggan',
                                      'Pemilik_Unit,_Skema_Bagi_Hasil': 'Pemilik_Unit', 
                                      'Unnamed:_23': 'Skema_Bagi_Hasil'})


In [78]:
#Combine data from 2 datasets
data = pd.concat([data_2017,data_2018])

print(f"Transaksi pertama: {min(data.Tanggal_Pesanan)}")
print(f"Transaksi terakhir: {max(data.Tanggal_Pesanan)}")
print(f"Jumlah transaksi: {len(data.Nomor.unique())}")
print(f"Jumlah unit disewakan: {len(data.Kode_Unit.unique())}")

Transaksi pertama: 2017-05-12 00:00:00
Transaksi terakhir: 2020-01-24 00:00:00
Jumlah transaksi: 14949
Jumlah unit disewakan: 2134


In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34855 entries, 0 to 27212
Data columns (total 12 columns):
Tanggal_Pesanan     34822 non-null datetime64[ns]
Nomor               34822 non-null object
Komponen_Biaya      34822 non-null object
Harga               34822 non-null float64
Kode_Unit           18062 non-null object
Mulai_Sewa          18062 non-null datetime64[ns]
Selesai_Sewa        18062 non-null datetime64[ns]
Durasi_(hari)       18062 non-null float64
Nama_Pelanggan      18062 non-null object
Alamat_Pelanggan    18062 non-null object
Pemilik_Unit        18058 non-null object
Skema_Bagi_Hasil    18019 non-null object
dtypes: datetime64[ns](3), float64(2), object(7)
memory usage: 3.5+ MB


In [52]:
data.Kode_Unit.value_counts()

HDII342                        39
FLA1                           34
HDII18                         33
UT30                           33
DN31                           32
                               ..
GIN3                            1
TK1                             1
COCOLATTET80AN-DKARTGX-1904     1
MGP2                            1
HDII435                         1
Name: Kode_Unit, Length: 2133, dtype: int64